In [1]:
"""
========================================================================
HOTEL REVENUE ANALYTICS - SQL STAR SCHEMA DATABASE
========================================================================
Skills: SQL, Database Design, Star Schema, ETL
========================================================================
"""

import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("🗄️  BUILDING SQL STAR SCHEMA DATABASE")
print("=" * 70)

🗄️  BUILDING SQL STAR SCHEMA DATABASE


In [5]:
# ========================================================================
# WHAT IS A STAR SCHEMA?
# ========================================================================
print("""
📚 STAR SCHEMA EXPLANATION:

A star schema is a database design with:
- 1 FACT TABLE (center) - Contains metrics like revenue, bookings
- Multiple DIMENSION TABLES (points) - Contains descriptive data

Our Schema:
    
    dim_date ──┐
               │
    dim_room ──┼──→ FACT_BOOKINGS (center)
               │
  dim_segment ─┘

Benefits:
✓ Fast queries (fewer joins needed)
✓ Easy to understand
✓ Industry standard for BI/reporting
✓ Optimized for analytics
""")


📚 STAR SCHEMA EXPLANATION:

A star schema is a database design with:
- 1 FACT TABLE (center) - Contains metrics like revenue, bookings
- Multiple DIMENSION TABLES (points) - Contains descriptive data

Our Schema:
    
    dim_date ──┐
               │
    dim_room ──┼──→ FACT_BOOKINGS (center)
               │
  dim_segment ─┘

Benefits:
✓ Fast queries (fewer joins needed)
✓ Easy to understand
✓ Industry standard for BI/reporting
✓ Optimized for analytics



In [6]:
# ========================================================================
# STEP 1: CONNECT TO DATABASE
# ========================================================================
print("\n📁 STEP 1: Creating database connection...")
print("=" * 70)

# Database file path
db_path = '../data/hotel_revenue.db'

# Connect to SQLite (creates file if doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"✅ Connected to: {db_path}")

# Check if database already has tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
existing_tables = cursor.fetchall()

if existing_tables:
    print(f"⚠️  Found {len(existing_tables)} existing tables - will recreate them")
else:
    print("✅ Database is empty - ready for setup!")


📁 STEP 1: Creating database connection...
✅ Connected to: ../data/hotel_revenue.db
✅ Database is empty - ready for setup!


In [7]:
# ========================================================================
# STEP 2: DROP EXISTING TABLES (FRESH START)
# ========================================================================
print("\n🧹 STEP 2: Cleaning database (fresh start)...")
print("=" * 70)

# Drop tables in correct order (fact first, then dimensions)
tables_to_drop = ['fact_bookings', 'dim_date', 'dim_room', 'dim_segment']

for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    print(f"✅ Dropped table: {table}")

conn.commit()
print("\n✅ Database cleaned!")


🧹 STEP 2: Cleaning database (fresh start)...
✅ Dropped table: fact_bookings
✅ Dropped table: dim_date
✅ Dropped table: dim_room
✅ Dropped table: dim_segment

✅ Database cleaned!


In [8]:
# ========================================================================
# STEP 3: CREATE DIMENSION TABLES
# ========================================================================
print("\n📐 STEP 3: Creating dimension tables...")
print("=" * 70)

# -------------------------
# Dimension 1: Date
# -------------------------
print("\n1️⃣ Creating dim_date...")

create_date = """
CREATE TABLE dim_date (
    date_key INTEGER PRIMARY KEY,
    date DATE NOT NULL,
    month INTEGER,
    month_name TEXT,
    quarter INTEGER,
    year INTEGER,
    is_weekend INTEGER
)
"""
cursor.execute(create_date)
print("   ✅ dim_date created")
print("      • date_key: Unique ID (YYYYMMDD format)")
print("      • date: Actual date")
print("      • month, quarter, year: For time analysis")
print("      • is_weekend: Flag for weekend analysis")

# -------------------------
# Dimension 2: Room
# -------------------------
print("\n2️⃣ Creating dim_room...")

create_room = """
CREATE TABLE dim_room (
    room_key INTEGER PRIMARY KEY AUTOINCREMENT,
    room_type TEXT NOT NULL,
    base_price REAL
)
"""
cursor.execute(create_room)
print("   ✅ dim_room created")
print("      • room_key: Auto-incrementing unique ID")
print("      • room_type: Room category (A, B, C, etc.)")
print("      • base_price: Median price for this room type")

# -------------------------
# Dimension 3: Market Segment
# -------------------------
print("\n3️⃣ Creating dim_segment...")

create_segment = """
CREATE TABLE dim_segment (
    segment_key INTEGER PRIMARY KEY AUTOINCREMENT,
    market_segment TEXT NOT NULL
)
"""
cursor.execute(create_segment)
print("   ✅ dim_segment created")
print("      • segment_key: Auto-incrementing unique ID")
print("      • market_segment: Customer type (Corporate, Online TA, etc.)")

conn.commit()
print("\n✅ All 3 dimension tables created!")


📐 STEP 3: Creating dimension tables...

1️⃣ Creating dim_date...
   ✅ dim_date created
      • date_key: Unique ID (YYYYMMDD format)
      • date: Actual date
      • month, quarter, year: For time analysis
      • is_weekend: Flag for weekend analysis

2️⃣ Creating dim_room...
   ✅ dim_room created
      • room_key: Auto-incrementing unique ID
      • room_type: Room category (A, B, C, etc.)
      • base_price: Median price for this room type

3️⃣ Creating dim_segment...
   ✅ dim_segment created
      • segment_key: Auto-incrementing unique ID
      • market_segment: Customer type (Corporate, Online TA, etc.)

✅ All 3 dimension tables created!


In [9]:
# ========================================================================
# STEP 4: CREATE FACT TABLE
# ========================================================================
print("\n📊 STEP 4: Creating fact table...")
print("=" * 70)

create_fact = """
CREATE TABLE fact_bookings (
    booking_key INTEGER PRIMARY KEY AUTOINCREMENT,
    booking_id TEXT NOT NULL,
    
    -- Foreign keys to dimensions
    date_key INTEGER,
    room_key INTEGER,
    segment_key INTEGER,
    
    -- Measures (metrics we want to analyze)
    adr REAL,
    nights INTEGER,
    guests INTEGER,
    revenue REAL,
    is_cancelled INTEGER,
    
    -- Relationships to dimension tables
    FOREIGN KEY (date_key) REFERENCES dim_date(date_key),
    FOREIGN KEY (room_key) REFERENCES dim_room(room_key),
    FOREIGN KEY (segment_key) REFERENCES dim_segment(segment_key)
)
"""
cursor.execute(create_fact)

print("✅ fact_bookings created")
print("\n📋 Fact Table Structure:")
print("   • booking_key: Unique ID for each booking")
print("   • booking_id: Business booking number")
print("   • Foreign Keys: Links to dim_date, dim_room, dim_segment")
print("   • Measures: adr, nights, guests, revenue, is_cancelled")

conn.commit()
print("\n✅ Fact table created!")


📊 STEP 4: Creating fact table...
✅ fact_bookings created

📋 Fact Table Structure:
   • booking_key: Unique ID for each booking
   • booking_id: Business booking number
   • Foreign Keys: Links to dim_date, dim_room, dim_segment
   • Measures: adr, nights, guests, revenue, is_cancelled

✅ Fact table created!


In [10]:
# ========================================================================
# STEP 5: CREATE INDEXES FOR PERFORMANCE
# ========================================================================
print("\n⚡ STEP 5: Creating indexes for fast queries...")
print("=" * 70)

print("\n💡 What are indexes?")
print("   Like a book's index - helps find data quickly")
print("   Without index: Search all 79,000 rows")
print("   With index: Jump directly to relevant rows")

# Create indexes on frequently queried columns
indexes = [
    ("idx_date", "fact_bookings", "date_key"),
    ("idx_room", "fact_bookings", "room_key"),
    ("idx_segment", "fact_bookings", "segment_key"),
    ("idx_cancelled", "fact_bookings", "is_cancelled")
]

for idx_name, table, column in indexes:
    cursor.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
    print(f"✅ Created index: {idx_name} on {table}({column})")

conn.commit()
print(f"\n✅ Created {len(indexes)} indexes - queries will be FAST!")


⚡ STEP 5: Creating indexes for fast queries...

💡 What are indexes?
   Like a book's index - helps find data quickly
   Without index: Search all 79,000 rows
   With index: Jump directly to relevant rows
✅ Created index: idx_date on fact_bookings(date_key)
✅ Created index: idx_room on fact_bookings(room_key)
✅ Created index: idx_segment on fact_bookings(segment_key)
✅ Created index: idx_cancelled on fact_bookings(is_cancelled)

✅ Created 4 indexes - queries will be FAST!


In [11]:
# ========================================================================
# STEP 6: LOAD CLEANED DATA
# ========================================================================
print("\n📥 STEP 6: Loading cleaned data...")
print("=" * 70)

# Load the cleaned CSV
df = pd.read_csv('../data/hotel_bookings_cleaned.csv')
df['arrival_date'] = pd.to_datetime(df['arrival_date'])

print(f"✅ Loaded {len(df):,} bookings from CSV")
print(f"   Date range: {df['arrival_date'].min().date()} to {df['arrival_date'].max().date()}")
print(f"   Columns: {len(df.columns)}")
print(f"   Total revenue: €{df['total_revenue'].sum():,.2f}")


📥 STEP 6: Loading cleaned data...
✅ Loaded 97,391 bookings from CSV
   Date range: 2016-01-01 to 2017-08-31
   Columns: 38
   Total revenue: €21,484,255.10


In [12]:
# ========================================================================
# STEP 7: POPULATE DIMENSION TABLES
# ========================================================================
print("\n📊 STEP 7: Populating dimension tables...")
print("=" * 70)

# -------------------------
# Populate: dim_date
# -------------------------
print("\n📅 Populating dim_date...")

# Get all unique dates
dates = df['arrival_date'].unique()

date_data = []
for date in sorted(dates):
    dt = pd.Timestamp(date)
    date_data.append({
        'date_key': int(dt.strftime('%Y%m%d')),  # e.g., 20160715
        'date': dt.strftime('%Y-%m-%d'),         # e.g., 2016-07-15
        'month': dt.month,
        'month_name': dt.month_name(),
        'quarter': dt.quarter,
        'year': dt.year,
        'is_weekend': 1 if dt.dayofweek >= 5 else 0
    })

# Convert to DataFrame and load
date_df = pd.DataFrame(date_data)
date_df.to_sql('dim_date', conn, if_exists='append', index=False)

print(f"   ✅ Loaded {len(date_df):,} dates")
print(f"      Date range: {date_df['date'].min()} to {date_df['date'].max()}")
print(f"      Sample date_keys: {date_df['date_key'].head(3).tolist()}")


📊 STEP 7: Populating dimension tables...

📅 Populating dim_date...
   ✅ Loaded 609 dates
      Date range: 2016-01-01 to 2017-08-31
      Sample date_keys: [20160101, 20160102, 20160103]


In [13]:
# -------------------------
# Populate: dim_room
# -------------------------
print("\n🛏️  Populating dim_room...")

# Get unique room types
room_types = df['reserved_room_type'].unique()

room_data = []
for room_type in sorted(room_types):
    # Calculate base price as median ADR for this room type
    room_bookings = df[df['reserved_room_type'] == room_type]
    base_price = room_bookings['adr'].median()
    
    room_data.append({
        'room_type': room_type,
        'base_price': round(base_price, 2)
    })

# Load to database
room_df = pd.DataFrame(room_data)
room_df.to_sql('dim_room', conn, if_exists='append', index=False)

print(f"   ✅ Loaded {len(room_df)} room types")
print("\n   📋 Room Types & Prices:")
for _, row in room_df.iterrows():
    print(f"      • {row['room_type']}: €{row['base_price']:.2f}")


🛏️  Populating dim_room...
   ✅ Loaded 9 room types

   📋 Room Types & Prices:
      • A: €90.00
      • B: €90.46
      • C: €168.00
      • D: €123.30
      • E: €115.00
      • F: €179.10
      • G: €179.00
      • H: €185.00
      • P: €0.00


In [14]:
# -------------------------
# Populate: dim_segment
# -------------------------
print("\n👥 Populating dim_segment...")

# Get unique market segments
segments = df['market_segment'].unique()

segment_data = [{'market_segment': seg} for seg in sorted(segments)]

# Load to database
segment_df = pd.DataFrame(segment_data)
segment_df.to_sql('dim_segment', conn, if_exists='append', index=False)

print(f"   ✅ Loaded {len(segment_df)} market segments")
print("\n   📋 Market Segments:")
for seg in sorted(segments):
    count = (df['market_segment'] == seg).sum()
    pct = (count / len(df)) * 100
    print(f"      • {seg:20s}: {count:6,} bookings ({pct:5.1f}%)")

print("\n✅ All dimension tables populated!")


👥 Populating dim_segment...
   ✅ Loaded 7 market segments

   📋 Market Segments:
      • Aviation            :    237 bookings (  0.2%)
      • Complementary       :    578 bookings (  0.6%)
      • Corporate           :  4,124 bookings (  4.2%)
      • Direct              : 10,292 bookings ( 10.6%)
      • Groups              : 13,710 bookings ( 14.1%)
      • Offline TA/TO       : 18,138 bookings ( 18.6%)
      • Online TA           : 50,312 bookings ( 51.7%)

✅ All dimension tables populated!


In [15]:
# ========================================================================
# STEP 8: PREPARE FACT TABLE DATA
# ========================================================================
print("\n📊 STEP 8: Preparing fact table data...")
print("=" * 70)

# -------------------------
# Get dimension keys
# -------------------------
print("\n🔗 Retrieving dimension keys...")

room_keys = pd.read_sql("SELECT * FROM dim_room", conn)
segment_keys = pd.read_sql("SELECT * FROM dim_segment", conn)

print(f"   ✅ Retrieved {len(room_keys)} room keys")
print(f"   ✅ Retrieved {len(segment_keys)} segment keys")

# -------------------------
# Merge keys to main dataframe
# -------------------------
print("\n🔗 Joining dimension keys to booking data...")

df = df.merge(room_keys, left_on='reserved_room_type', right_on='room_type', how='left')
df = df.merge(segment_keys, on='market_segment', how='left')

print("   ✅ Joined room keys")
print("   ✅ Joined segment keys")

# -------------------------
# Create booking IDs and date keys
# -------------------------
print("\n🆔 Creating booking IDs...")

df['booking_id'] = 'BK' + (df.index + 1).astype(str).str.zfill(6)
print(f"   ✅ Created {len(df):,} booking IDs (format: BK000001)")

print("\n📅 Creating date keys...")
df['date_key'] = pd.to_datetime(df['arrival_date']).dt.strftime('%Y%m%d').astype(int)
print(f"   ✅ Created date keys (format: YYYYMMDD)")

print("\n✅ Fact data prepared!")


📊 STEP 8: Preparing fact table data...

🔗 Retrieving dimension keys...
   ✅ Retrieved 9 room keys
   ✅ Retrieved 7 segment keys

🔗 Joining dimension keys to booking data...
   ✅ Joined room keys
   ✅ Joined segment keys

🆔 Creating booking IDs...
   ✅ Created 97,391 booking IDs (format: BK000001)

📅 Creating date keys...
   ✅ Created date keys (format: YYYYMMDD)

✅ Fact data prepared!


In [16]:
# ========================================================================
# STEP 9: LOAD FACT TABLE
# ========================================================================
print("\n💾 STEP 9: Loading fact table...")
print("=" * 70)

# Select columns for fact table
fact_data = df[[
    'booking_id', 'date_key', 'room_key', 'segment_key',
    'adr', 'total_nights', 'total_guests', 'total_revenue', 'is_canceled'
]]

# Rename columns to match fact table schema
fact_data.columns = [
    'booking_id', 'date_key', 'room_key', 'segment_key',
    'adr', 'nights', 'guests', 'revenue', 'is_cancelled'
]

# Remove any rows with missing keys
fact_data = fact_data.dropna(subset=['room_key', 'segment_key'])

print(f"📦 Prepared {len(fact_data):,} fact records")

# Load to database
fact_data.to_sql('fact_bookings', conn, if_exists='append', index=False)

print(f"✅ Loaded {len(fact_data):,} bookings to fact_bookings table!")
print("\n🎉 Database loading complete!")


💾 STEP 9: Loading fact table...
📦 Prepared 97,391 fact records
✅ Loaded 97,391 bookings to fact_bookings table!

🎉 Database loading complete!


In [17]:
# ========================================================================
# STEP 10: VERIFY DATABASE
# ========================================================================
print("\n✅ STEP 10: Verifying database...")
print("=" * 70)

# Get row counts for all tables
print("\n📊 TABLE ROW COUNTS:")
print("-" * 70)

tables = ['dim_date', 'dim_room', 'dim_segment', 'fact_bookings']
for table in tables:
    count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table}", conn).iloc[0]['cnt']
    print(f"   {table:20s}: {count:>8,} rows")

print("\n✅ All tables populated successfully!")


✅ STEP 10: Verifying database...

📊 TABLE ROW COUNTS:
----------------------------------------------------------------------
   dim_date            :      609 rows
   dim_room            :        9 rows
   dim_segment         :        7 rows
   fact_bookings       :   97,391 rows

✅ All tables populated successfully!


In [18]:
# ========================================================================
# STEP 11: TEST QUERIES
# ========================================================================
print("\n🧪 STEP 11: Testing SQL queries...")
print("=" * 70)

print("\n📊 Test Query 1: Revenue by Room Type")
print("-" * 70)

query = """
SELECT 
    r.room_type,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as total_revenue,
    ROUND(AVG(f.adr), 2) as avg_adr
FROM fact_bookings f
JOIN dim_room r ON f.room_key = r.room_key
WHERE f.is_cancelled = 0
GROUP BY r.room_type
ORDER BY total_revenue DESC
LIMIT 5
"""

result = pd.read_sql(query, conn)
print(result.to_string(index=False))


🧪 STEP 11: Testing SQL queries...

📊 Test Query 1: Revenue by Room Type
----------------------------------------------------------------------
room_type  bookings  total_revenue  avg_adr
        A     41940    11745609.37    90.88
        D     11294     5229112.98   119.44
        E      3871     2045045.11   120.46
        F      1700      987631.59   163.38
        G      1139      755664.93   172.61


In [19]:
print("\n\n📊 Test Query 2: Monthly Revenue Trend")
print("-" * 70)

query = """
SELECT 
    d.year,
    d.month_name,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
WHERE f.is_cancelled = 0
GROUP BY d.year, d.month, d.month_name
ORDER BY d.year, d.month
LIMIT 6
"""

result = pd.read_sql(query, conn)
print(result.to_string(index=False))



📊 Test Query 2: Monthly Revenue Trend
----------------------------------------------------------------------
 year month_name  bookings    revenue
 2016    January      1691  264521.38
 2016   February      2554  484170.72
 2016      March      3347  767337.42
 2016      April      3367  896591.38
 2016        May      3563 1073277.63
 2016       June      3196 1144800.38


In [20]:
print("\n\n📊 Test Query 3: Market Segment Performance")
print("-" * 70)

query = """
SELECT 
    s.market_segment,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue,
    ROUND(AVG(f.adr), 2) as avg_adr
FROM fact_bookings f
JOIN dim_segment s ON f.segment_key = s.segment_key
WHERE f.is_cancelled = 0
GROUP BY s.market_segment
ORDER BY revenue DESC
"""

result = pd.read_sql(query, conn)
print(result.to_string(index=False))

print("\n✅ All test queries successful!")



📊 Test Query 3: Market Segment Performance
----------------------------------------------------------------------
market_segment  bookings     revenue  avg_adr
     Online TA     31218 12023118.84   115.56
 Offline TA/TO     11895  4302157.09    84.10
        Direct      8709  3291609.26   115.28
        Groups      5456  1365693.04    77.91
     Corporate      3351   426872.98    68.42
      Aviation       185    70868.36    99.48
 Complementary       496     3935.53     3.23

✅ All test queries successful!


In [22]:
# ========================================================================
# FINAL SUMMARY
# ========================================================================
print("\n" + "=" * 70)
print("🎉 SQL DATABASE COMPLETE!")
print("=" * 70)

print(f"""
✅ WHAT WE BUILT:

Star Schema Database with:
- 3 Dimension Tables (date, room, segment)
- 1 Fact Table (bookings with metrics)
- 4 Indexes (for fast queries)
- {len(fact_data):,} booking records loaded

✅ SKILLS DEMONSTRATED:

- Database design (star schema)
- SQL table creation
- Foreign key relationships
- Index optimization
- ETL process (Extract, Transform, Load)
- Data modeling


✅ DATABASE FILE:

Location: {db_path}

""")

# Close connection
conn.close()
print("\n✅ Database connection closed")
print("=" * 70)


🎉 SQL DATABASE COMPLETE!

✅ WHAT WE BUILT:

Star Schema Database with:
- 3 Dimension Tables (date, room, segment)
- 1 Fact Table (bookings with metrics)
- 4 Indexes (for fast queries)
- 97,391 booking records loaded

✅ SKILLS DEMONSTRATED:

- Database design (star schema)
- SQL table creation
- Foreign key relationships
- Index optimization
- ETL process (Extract, Transform, Load)
- Data modeling


✅ DATABASE FILE:

Location: ../data/hotel_revenue.db



✅ Database connection closed
